In [31]:
import os
import numpy as np
import pandas as pd
import torch
import mne

In [32]:
# --------------------------------------------------------------------------------
# 1) Настройки
# --------------------------------------------------------------------------------

In [33]:
BIDS_ROOT = "/Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504"  # <-- Укажите путь к корню датасета (где sub-XXX и participants.tsv)
OUT_FILE = "my_eeg_dataset.pt"  # <-- Итоговый файл с (X, y)
PARTICIPANTS_TSV = "participants.tsv"

# Частоты фильтра
L_FREQ, H_FREQ = 0.5, 45.0  # Обновляем фильтр на 0.5–45 Гц согласно описанию датасета

# Настройки ICA
DO_ICA = True
ICA_NCOMP = 19  # Число компонентов равно количеству каналов

# Окна
WINDOW_SEC = 5.0  # Используем 5-секундные окна (2500 точек)
STEP_SEC = 2.0  # Шаг окна для наложения

# Уровни квантования
AMP_QUANT_LEVELS = 512  # Улучшенная дискретизация амплитуд

# Маппинг групп (A, C, F) -> метки
# A: Alzheimer, C: Control, F: "прочие" (например, Mild Cognitive Impairment и т.п.)
group_map = {
    'A': 1,
    'F': 2,
    'C': 0
}


In [34]:
# --------------------------------------------------------------------------------
# 2) Функции
# --------------------------------------------------------------------------------

In [35]:
def amplitude_quantization(seg, levels=256):
    """
    Упрощённая скалярная квантизация амплитуд.
    seg: (C, T) numpy array
    """
    seg_flat = seg.flatten()
    val_min, val_max = seg_flat.min(), seg_flat.max()
    if abs(val_min - val_max) < 1e-12:
        return np.zeros_like(seg_flat, dtype=int)
    seg_norm = (seg_flat - val_min) / (val_max - val_min)
    seg_q = np.floor(seg_norm * levels).astype(int)
    seg_q = np.clip(seg_q, 0, levels - 1)
    return seg_q


def process_one_subject(set_fname, l_freq=0.5, h_freq=45.0, do_ica=True, ica_ncomp=19):
    """
    Открытие и предварительная обработка EEG-данных.
    """
    print(f"Loading: {set_fname}")
    raw = mne.io.read_raw_eeglab(set_fname, preload=True)

    # Применение фильтрации
    print(f"Filtering {l_freq}–{h_freq} Hz")
    raw.filter(l_freq=l_freq, h_freq=h_freq)
    raw.set_eeg_reference(['A1', 'A2'])  # Перереференцирование

    # Установка монтажной схемы 10-20
    montage = mne.channels.make_standard_montage('standard_1020')
    raw.set_montage(montage)

    if do_ica:
        print(f"Applying ICA with {ica_ncomp} components.")
        ica = mne.preprocessing.ICA(n_components=ica_ncomp, random_state=42)
        ica.fit(raw)
        raw = ica.apply(raw, exclude=[0, 1])  # Исключаем компоненты, соответствующие артефактам

    return raw



def segment_and_quantize(raw, win_sec=2.0, step_sec=1.0, levels=256):
    """
    Нарезка на окна + квантование.
    """
    sfreq = raw.info['sfreq']
    data = raw.get_data()  # (C, N)
    n_times = data.shape[1]

    win_samps = int(win_sec * sfreq)
    step_samps = int(step_sec * sfreq)

    segments = []
    start = 0
    while (start + win_samps) <= n_times:
        seg = data[:, start:start + win_samps]  # (C, T)
        seg_q = amplitude_quantization(seg, levels=levels)
        segments.append(seg_q)
        start += step_samps
    return segments


In [36]:
# --------------------------------------------------------------------------------
# 3) Основной скрипт
# --------------------------------------------------------------------------------

In [37]:
def main():
    part_tsv = os.path.join(BIDS_ROOT, PARTICIPANTS_TSV)
    if not os.path.isfile(part_tsv):
        raise FileNotFoundError(f"Не найден {part_tsv}")
    participants_df = pd.read_csv(part_tsv, sep='\t')

    all_X = []
    all_y = []

    for idx, row in participants_df.iterrows():
        subj_id = row['participant_id']
        group = row['Group']
        if group not in group_map:
            print(f"Пропускаем {subj_id}: неизвестная группа '{group}'.")
            continue
        label = group_map[group]

        sub_folder = os.path.join(BIDS_ROOT, "derivatives", subj_id, "eeg")
        task_list = ["eyesclosed", "eyesopen"]

        for task in task_list:
            set_fname = os.path.join(sub_folder, f"{subj_id}_task-{task}_eeg.set")
            if not os.path.isfile(set_fname):
                print(f"Нет файла: {set_fname}, пропускаем задачу {task}.")
                continue

            try:
                raw = process_one_subject(
                    set_fname, l_freq=L_FREQ, h_freq=H_FREQ, do_ica=DO_ICA, ica_ncomp=ICA_NCOMP
                )
                seg_list = segment_and_quantize(raw, win_sec=WINDOW_SEC, step_sec=STEP_SEC, levels=AMP_QUANT_LEVELS)

                for seg_q in seg_list:
                    all_X.append(seg_q)
                    all_y.append(label)

                print(f"{subj_id}, {task}: {len(seg_list)} окон. Группа={group} -> label={label}")
            except Exception as ex:
                print(f"Ошибка subj={subj_id}, task={task}: {ex}")
                continue


    if len(all_X) == 0:
        print("Нет данных для сохранения. all_X пуст.")
        return

    X_array = np.stack(all_X, axis=0)
    X_tensor = torch.from_numpy(X_array).long()
    y_tensor = torch.tensor(all_y, dtype=torch.long)

    print(f"\nИтого всего окон: {X_tensor.shape[0]}")
    print(f"Размер одного сегмента: {X_tensor.shape[1]} (C*T)")
    print("Пример меток (y):", set(y_tensor.tolist()))

    out_path = os.path.join(BIDS_ROOT, OUT_FILE)
    torch.save((X_tensor, y_tensor), out_path)
    print(f"\nСохранено: {out_path}")


if __name__ == "__main__":
    main()


Loading: /Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-001/eeg/sub-001_task-eyesclosed_eeg.set
Ошибка subj=/Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-001/eeg/sub-001_task-eyesclosed_eeg.set: отсутствуют необходимые каналы ['A1', 'A2']
Ошибка subj=sub-001, task=eyesclosed: 'NoneType' object has no attribute 'info'
Нет файла: /Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-001/eeg/sub-001_task-eyesopen_eeg.set, пропускаем задачу eyesopen.
Loading: /Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-002/eeg/sub-002_task-eyesclosed_eeg.set
Ошибка subj=/Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-002/eeg/sub-002_task-eyesclosed_eeg.set: отсутствуют необходимые каналы ['A1', 'A2']
Ошибка subj=sub-002, task=eyesclosed: 'NoneType' object has no attribute 'info'
Нет файла: /Users/taniyashuba/PycharmProjects

/var/folders/87/d35_l8056xbbpvj6g4knhvsm0000gn/T/ipykernel_8684/2802821301.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_fname, preload=True)
/var/folders/87/d35_l8056xbbpvj6g4knhvsm0000gn/T/ipykernel_8684/2802821301.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_fname, preload=True)
/var/folders/87/d35_l8056xbbpvj6g4knhvsm0000gn/T/ipykernel_8684/2802821301.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_fname, preload=True)
/var/folders/87/d35_l8056xbbpvj6g4knhvsm0000gn/T/ipykernel_8684/2802821301.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data 

Ошибка subj=/Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-006/eeg/sub-006_task-eyesclosed_eeg.set: отсутствуют необходимые каналы ['A1', 'A2']
Ошибка subj=sub-006, task=eyesclosed: 'NoneType' object has no attribute 'info'
Нет файла: /Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-006/eeg/sub-006_task-eyesopen_eeg.set, пропускаем задачу eyesopen.
Loading: /Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-007/eeg/sub-007_task-eyesclosed_eeg.set
Ошибка subj=/Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-007/eeg/sub-007_task-eyesclosed_eeg.set: отсутствуют необходимые каналы ['A1', 'A2']
Ошибка subj=sub-007, task=eyesclosed: 'NoneType' object has no attribute 'info'
Нет файла: /Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-007/eeg/sub-007_task-eyesopen_eeg.set, пропускаем задачу eyesopen.
Loading: /Users/

/var/folders/87/d35_l8056xbbpvj6g4knhvsm0000gn/T/ipykernel_8684/2802821301.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_fname, preload=True)
/var/folders/87/d35_l8056xbbpvj6g4knhvsm0000gn/T/ipykernel_8684/2802821301.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_fname, preload=True)
/var/folders/87/d35_l8056xbbpvj6g4knhvsm0000gn/T/ipykernel_8684/2802821301.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_fname, preload=True)
/var/folders/87/d35_l8056xbbpvj6g4knhvsm0000gn/T/ipykernel_8684/2802821301.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data 

Ошибка subj=/Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-010/eeg/sub-010_task-eyesclosed_eeg.set: отсутствуют необходимые каналы ['A1', 'A2']
Ошибка subj=sub-010, task=eyesclosed: 'NoneType' object has no attribute 'info'
Нет файла: /Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-010/eeg/sub-010_task-eyesopen_eeg.set, пропускаем задачу eyesopen.
Loading: /Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-011/eeg/sub-011_task-eyesclosed_eeg.set
Ошибка subj=/Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-011/eeg/sub-011_task-eyesclosed_eeg.set: отсутствуют необходимые каналы ['A1', 'A2']
Ошибка subj=sub-011, task=eyesclosed: 'NoneType' object has no attribute 'info'
Нет файла: /Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-011/eeg/sub-011_task-eyesopen_eeg.set, пропускаем задачу eyesopen.
Loading: /Users/

...

/var/folders/87/d35_l8056xbbpvj6g4knhvsm0000gn/T/ipykernel_8684/2802821301.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_fname, preload=True)
/var/folders/87/d35_l8056xbbpvj6g4knhvsm0000gn/T/ipykernel_8684/2802821301.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_fname, preload=True)
/var/folders/87/d35_l8056xbbpvj6g4knhvsm0000gn/T/ipykernel_8684/2802821301.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_fname, preload=True)
/var/folders/87/d35_l8056xbbpvj6g4knhvsm0000gn/T/ipykernel_8684/2802821301.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data 

Ошибка subj=/Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-076/eeg/sub-076_task-eyesclosed_eeg.set: отсутствуют необходимые каналы ['A1', 'A2']
Ошибка subj=sub-076, task=eyesclosed: 'NoneType' object has no attribute 'info'
Нет файла: /Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-076/eeg/sub-076_task-eyesopen_eeg.set, пропускаем задачу eyesopen.
Loading: /Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-077/eeg/sub-077_task-eyesclosed_eeg.set
Ошибка subj=/Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-077/eeg/sub-077_task-eyesclosed_eeg.set: отсутствуют необходимые каналы ['A1', 'A2']
Ошибка subj=sub-077, task=eyesclosed: 'NoneType' object has no attribute 'info'
Нет файла: /Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-077/eeg/sub-077_task-eyesopen_eeg.set, пропускаем задачу eyesopen.
Loading: /Users/

/var/folders/87/d35_l8056xbbpvj6g4knhvsm0000gn/T/ipykernel_8684/2802821301.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_fname, preload=True)
/var/folders/87/d35_l8056xbbpvj6g4knhvsm0000gn/T/ipykernel_8684/2802821301.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_fname, preload=True)
/var/folders/87/d35_l8056xbbpvj6g4knhvsm0000gn/T/ipykernel_8684/2802821301.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_fname, preload=True)
/var/folders/87/d35_l8056xbbpvj6g4knhvsm0000gn/T/ipykernel_8684/2802821301.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data 

Loading: /Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-081/eeg/sub-081_task-eyesclosed_eeg.set
Ошибка subj=/Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-081/eeg/sub-081_task-eyesclosed_eeg.set: отсутствуют необходимые каналы ['A1', 'A2']
Ошибка subj=sub-081, task=eyesclosed: 'NoneType' object has no attribute 'info'
Нет файла: /Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-081/eeg/sub-081_task-eyesopen_eeg.set, пропускаем задачу eyesopen.
Loading: /Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-082/eeg/sub-082_task-eyesclosed_eeg.set
Ошибка subj=/Users/taniyashuba/PycharmProjects/eeg-transformer-gnn/data/ds004504/derivatives/sub-082/eeg/sub-082_task-eyesclosed_eeg.set: отсутствуют необходимые каналы ['A1', 'A2']
Ошибка subj=sub-082, task=eyesclosed: 'NoneType' object has no attribute 'info'
Нет файла: /Users/taniyashuba/PycharmProjects

/var/folders/87/d35_l8056xbbpvj6g4knhvsm0000gn/T/ipykernel_8684/2802821301.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_fname, preload=True)
/var/folders/87/d35_l8056xbbpvj6g4knhvsm0000gn/T/ipykernel_8684/2802821301.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_fname, preload=True)
/var/folders/87/d35_l8056xbbpvj6g4knhvsm0000gn/T/ipykernel_8684/2802821301.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_fname, preload=True)
/var/folders/87/d35_l8056xbbpvj6g4knhvsm0000gn/T/ipykernel_8684/2802821301.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data 

In [38]:
def process_one_subject(set_fname, l_freq=1., h_freq=40., do_ica=False, ica_ncomp=15):
    """
    Открывает .set-файл, фильтрует, (опционально) ICA, возвращает Raw (MNE).
    """
    print(f"Loading: {set_fname}")
    raw = mne.io.read_raw_eeglab(set_fname, preload=True)

    # Проверяем, присутствуют ли нужные каналы
    required_channels = ['A1', 'A2']
    missing_channels = [ch for ch in required_channels if ch not in raw.ch_names]

    if missing_channels:
        print(f"Ошибка subj={set_fname}: отсутствуют необходимые каналы {missing_channels}")
        return None

    print(f"Filtering {l_freq}–{h_freq} Hz")
    raw.filter(l_freq=l_freq, h_freq=h_freq)
    raw.set_eeg_reference(['A1', 'A2'], projection=False)

    if do_ica:
        print(f"Applying ICA with {ica_ncomp} components (but not excluding any comps).")
        ica = mne.preprocessing.ICA(n_components=ica_ncomp, random_state=42)
        ica.fit(raw)
        raw = ica.apply(raw)

    return raw
